In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup,NavigableString, Tag
import time
import requests
import re
import csv
from selenium.webdriver.common.by import By
import pandas as pd
import os

s = Service(r"/usr/local/bin/chromedriver") #使用chromedriver、selenium
driver = webdriver.Chrome(service=s)
    
driver.get("https://icard.ai/") #自動打開網頁（不要關）
    
time.sleep(10) #暫停
    
soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀

results = soup.find_all('div', class_='sc-fkouio-0 hniKmL') 
for result in results:
    
    ranking_title = [] #排行名
    card_name = [] #卡名
    card_link = [] #卡連結
    apply_link = [] #申請連結
    card_img = [] #卡圖片
    card_feature = [] #卡特色 
    
    title = result.find('h3').text
    if title == '2023最新熱門必辦信貸' or  title == '2023信用卡首刷禮推薦' or title == '2023下半年必辦信用卡':
        continue
        
    print(title)
    ranking_title.append(title) #排行名
    cards = result.find_all('div', class_='sc-fkouio-0 bkXcuC')#卡名
    for card in cards:
        name = card.find('div', class_='sc-fkouio-0 flkoQp')
        print(name.text)
        card_name.append(name.text)
        
        link = card.find('div', class_='sc-fkouio-0 eWyXAj')#卡連結
        if link != None:
            link = link.find('a').get('href')
            print('https://icard.ai'+link)
            card_link.append('https://icard.ai'+link)
        else:
            print('無')
            card_link.append('無')
            
        apply = card.find('div', class_='sc-fkouio-0 hTPJxD').find('a').get('href') #申請連結
        print('https://icard.ai'+apply)
        apply_link.append('https://icard.ai'+apply)
        
        c = card.find('div', class_='sc-fkouio-0 sc-17j05y-1 llqmnx kYmowd')#卡圖片
        if c != None:
            img = c.find('img',sizes='100vw').get('src')
            print('https://icard.ai'+img)
            card_img.append('https://icard.ai'+img)
        else:
            print('無')
            card_img.append('無')
            
        result1 =card.find('ul', class_='sc-fkouio-0 sc-1h42shb-2 cjrPow bBDGZz') #卡特色
        result2 = result1.find_all('span')
        text = []
        for re in result2:
            print(re.text)
            text.append(re.text)
        t = '\n'.join(text)
        card_feature.append(t) 
        print('\n')
    print('\n')
    
    cards_dict = {
        '卡名': card_name,
        #'卡連結': card_link,
        #'卡圖片': card_img,
        #'申請連結': apply_link,
        '卡片特色': card_feature    
    }
    
    df = pd.DataFrame(cards_dict)
    
    
    #創資料夾
    
    folder_name = 'icardai'+ranking_title[0]
    path = "/Users/takoyaki/Downloads/專題爬蟲/icardaicsv"  ##改掉資料夾路徑

    folder_path = os.path.join(path, folder_name)
    if not os.path.isdir(folder_path):
        os.makedirs(folder_path, mode=0o777)
    path0 = str(path+'/'+folder_name+"/")
    print(path0)
    df.to_csv(path0+('icardai'+ranking_title[0]+".csv"), index = False)
    
    #driver.close()
    
    #單卡頁面
    for i in range(len(card_name)):
        
        s = Service(r"/usr/local/bin/chromedriver") #使用chromedriver、selenium
        driver = webdriver.Chrome(service=s)
        driver.get(card_link[i]) #自動打開網頁（不要關）
        driver.fullscreen_window()
        time.sleep(3) #暫停
        soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀
        
    #卡名    
    
        result = soup.find('div', class_='sc-fkouio-0 kMjtwV')
        if result ==None:
            continue
        print('★'+result.text+'★')
        name = result.text #卡名
        
    #卡圖片
    
        result = soup.find('div',class_='sc-fkouio-0 sc-17j05y-1 llqmnx jHJsrR')
        if result !=None:
            result = result.find('img')
            img = 'https://icard.ai/'+result.get('src') #卡圖片
            print('卡圖片：\n'+img) 
        else:
            continue
    
        
        
        button= driver.find_element(By.CLASS_NAME, "fpOKLg") #年費
        button.click()
        time.sleep(2)
        
        soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀
        
    #年費彈出框
    
        title = [] #中標
        text1 = [] #內文
        card_apply_link =[] #申請連結
    
        results = soup.find_all('li', class_='sc-1ozxck3-1 llcEFQ')
        for result in results:
            title.append(result.find('div').text[:-1])
            text1.append(result.find('div',class_='sc-1ozxck3-2 eIFtwi').text)
        
        for i in range(len(title)):
            print(title[i])
            print(text1[i])
        
        result = soup.find('div', class_='sc-fkouio-0 iDmBLP')
        if result == None:
            card_apply_link.append('停止申辦')
        else:
            a = result.find('a').get('href')
            card_apply_link.append(a[7:])
            print('\n申請連結：\n'+ card_apply_link[0])
        
        button = driver.find_element(By.CLASS_NAME,'bzgOID') #年費彈出框關起來
        button.click()
        
    #優惠
    
        driver.execute_script("window.scrollBy(0,2400)","") 
        time.sleep(3) #暫停
        
        elements = driver.find_elements(By.CLASS_NAME, "clPheL")#展開
        if len(elements) > 0:
            element = elements[0]  # 取得找到的第一個元素
            element.click()  # 點擊元素
        
        driver.execute_script("window.scrollTo(0, 0);")#滑到頁面頂端
        time.sleep(3) #暫停
        driver.execute_script("window.scrollBy(0,800)","") #自動模擬滑鼠滾輪
        time.sleep(3) #暫停
        
        #用driver找到按鈕 並按下
        buttons = driver.find_elements(By.CLASS_NAME, "eOsKUS") #展開
        for button in buttons:
            if button.is_displayed() and button.is_enabled():
                button.click()
                driver.execute_script("window.scrollBy(0,200)","")
            else:
                #driver.execute_script("window.scrollBy(0,10)","")
                driver.execute_script("window.scrollBy(0,10)","")
                if button.is_displayed() and button.is_enabled():
                    button.click()
                else:
                    driver.execute_script("window.scrollBy(0,10)","")
                    if button.is_displayed() and button.is_enabled():
                        button.click()
                    else:
                        driver.execute_script("window.scrollBy(0,-30)","")
                        if button.is_displayed() and button.is_enabled():
                            button.click()
                        else:
                            driver.execute_script("window.scrollBy(0,50)","") 
                            button.click()
                driver.execute_script("window.scrollBy(0,200)","")
            
        time.sleep(3) #暫停    
        soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀
        #print(soup)
    
        space_list = [] #大標
        results = soup.find_all('div',class_='sc-fkouio-0 eynObR')
        for result1 in results:
            space_list.append(result1.text.strip()) #去前後空格
            #print(result1.text.strip())
        keyword='、'.join(space_list) #把大標當關鍵字
    
        title_list = [] #中標
        text_list = [] #內文
        results = soup.find_all('div',class_='sc-fkouio-0 aJYMs')
        for result1 in results:
            result2 = result1.find('div',class_='sc-fkouio-0 lgyuOQ')#中標
            title_list.append(result2.text)
            result3 = result1.find_all('li',class_='sc-1ozxck3-1 llcEFQ') #商家 詳情
            text = []
            for result4 in result3:
                text.append(result4.text)
                
            text='\n'.join(text)
            #print(type(text))
            text_list.append(text)
        
        for i in range(len(title_list)):
            print('☆'+title_list[i])
            print(text_list[i])
            print('\n')
        
        time.sleep(3) #暫停    
        driver.close()
    
    #裝進dict
    
        card_dict = {
            '卡名':name,
            '圖片':img,
            '申請連結':card_apply_link,
        }
        
        dict1=dict(zip(title, text1))
        dict2 = {
            '優惠標題':title_list,
            '優惠詳情':text_list
        }
        dict3 = {
            '關鍵字':keyword
        }
        
        card_dict.update(dict1)
        #card_dict.update(dict2)
        card_dict.update(dict3)
        
        #df = pd.DataFrame(card_dict)
        #path1 = "/Users/takoyaki/Downloads/專題爬蟲/icardaicsv/icardai"+ranking_title[0]+"/卡csv/"
        #if not os.path.isdir(path1):
        #    os.makedirs(path1, mode=0o777)
        #df.to_csv(path1+'icardai'+name+".csv", index = False)
        
        df = pd.DataFrame(dict2)
        path2 = "/Users/takoyaki/Downloads/專題爬蟲/icardaicsv/icardai"+ranking_title[0]+"/優惠csv/" ##改掉資料夾路徑
        if not os.path.isdir(path2):
            os.makedirs(path2, mode=0o777)
        df.to_csv(path2+'icardai'+name+"優惠.csv", index = False)
        
#driver.close()